In [1]:
import utils

	
 !"#%&'()*+,-./0123456789:<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_abcdefghijklmnopqrstuvwxyz{}ĆćčđŠšžНУабвгдезиклмнопрстухчшђјњћ…⁠✋❤﻿🏻🏼🏽👋👌👍👽💪😀😂😃😍😟🙂🙌🙏🤑🤘🤙🤝🤢🤣🤮🥰🥷🥹🧠🫂🫶
159
Step 0: train loss 5.1719, val loss 5.1660
Step 1000: train loss 2.0654, val loss 2.1860
Step 2000: train loss 1.8655, val loss 2.0280
Step 3000: train loss 1.7916, val loss 1.9716
Step 4000: train loss 1.8023, val loss 1.9515
Step 5000: train loss 1.7831, val loss 1.9248
Step 6000: train loss 1.7616, val loss 1.9299
Step 7000: train loss 1.7431, val loss 1.9130
Step 8000: train loss 1.7482, val loss 1.9074
Step 9000: train loss 1.7362, val loss 1.9303


In [2]:
utils.intuition_test('ee brate')

tensor([[67, 67,  2, 64, 80, 63, 82, 67]])
tensor([[67, 67,  2, 64, 80, 63, 82, 67]])
tensor([[67, 67,  2, 64, 80, 63, 82, 67,  1]])
tensor([[67,  2, 64, 80, 63, 82, 67,  1]])
tensor([[67, 67,  2, 64, 80, 63, 82, 67,  1, 59]])
tensor([[ 2, 64, 80, 63, 82, 67,  1, 59]])
tensor([[67, 67,  2, 64, 80, 63, 82, 67,  1, 59, 26]])
tensor([[64, 80, 63, 82, 67,  1, 59, 26]])
tensor([[67, 67,  2, 64, 80, 63, 82, 67,  1, 59, 26, 16]])
tensor([[80, 63, 82, 67,  1, 59, 26, 16]])
tensor([[67, 67,  2, 64, 80, 63, 82, 67,  1, 59, 26, 16, 18]])
tensor([[63, 82, 67,  1, 59, 26, 16, 18]])
tensor([[67, 67,  2, 64, 80, 63, 82, 67,  1, 59, 26, 16, 18, 16]])
tensor([[82, 67,  1, 59, 26, 16, 18, 16]])
tensor([[67, 67,  2, 64, 80, 63, 82, 67,  1, 59, 26, 16, 18, 16, 20]])
tensor([[67,  1, 59, 26, 16, 18, 16, 20]])
tensor([[67, 67,  2, 64, 80, 63, 82, 67,  1, 59, 26, 16, 18, 16, 20, 16]])
tensor([[ 1, 59, 26, 16, 18, 16, 20, 16]])
tensor([[67, 67,  2, 64, 80, 63, 82, 67,  1, 59, 26, 16, 18, 16, 20, 16, 19]])
ten